# **Tarjeta de datos 3**

### Hipótesis 3: "A mayor número de personas en un hogar es más probable que se invierta en energía renovable"

- Por un lado, se utiliza *tipo_nucleo_familiar.csv* para procesarlo y hacer una mejor agrupación de hogares en España. 
- Por otro lado, mediante *dispositivos_renovable.csv* se extrae el número de dispositivos que son de energía renovables en cada provincia.

- Con estos datos, se podrá estudiar la correlación que hay entre los diferentes núcleos famliares y el porcentaje de dispositivos renovables. Para ello, en primer lugar, habrá que realizar un mejor procesamiento de los datos, limpiando y redimensionando los datasets. Así, se conseguirá tener nuestro dataset gold para esta tarjeta de datos.

- El dataset final estará compuesto por índices en función del total, tanto para dispositivos de energía renovable como para cada tipo de núcleo familiar. De esta manera, se evita que el análisis no esté sesgado hacia provincias con mayor población. Así habrá comparaciones equitativas por los datos normalizados.

## **Carga de datos**

In [ ]:
# Importar librerías
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot

SILVER_DATA_PATH = os.path.join("..", "data/silver/")
GOLD_DATA_PATH = os.path.join("..", "data/gold/")

In [ ]:
# Dispositivos de energía renovable por provincia e ingresos.
dispositivos_renovable_df = pd.read_csv(SILVER_DATA_PATH+"dispositivos_renovable.csv", sep = ";", encoding = "latin")

#dataframe info.
dispositivos_renovable_df.info()

In [ ]:
# Cargar el DataFrame con data type str para evitar errores de lectura, luego se cambiará el tipo de dato si es necesario
tipo_nucleo_familiar_df = pd.read_csv(SILVER_DATA_PATH + "tipo_nucleo_familiar.csv", sep=";", encoding="latin")

# Ver la información del DataFrame
tipo_nucleo_familiar_df.info()


## **Procesamiento**

### tipo_nucleo_familiar.csv


In [ ]:
tipo_nucleo_familiar_df.head(13)

- Se ve que en el dataset tipo_nucleo_familar_df tiene valores iguales a cero en la columna 'Total'. Se comprueba que filas son y se eliminarán si no afectan a la composición y calidad de los datos. Pues no aportan interés.

In [ ]:
# Filas con valores 0 en la columna 'Total'
total_cero_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Total'] == 0]
print("Dimensión del dataframe con valores 0:", total_cero_df.shape)

print('\n',total_cero_df.head())

# Valores únicos de las columnas 'Tipo de núcleo familiar' y 'Número de hijos' para total_cero_df
valores_unicos = total_cero_df[['Tipo de núcleo familiar', 'Número de hijos']].drop_duplicates().values.tolist()
print('\n',valores_unicos)

- Se ve que en el dataframe las filas donde 'Total' es cero son aquellas que tienen la combinación: [['Padre sólo con hijos convivientes', '0 hijos conviviendo'], ['Madre sóla con hijos convivientes', '0 hijos conviviendo']]
    - Se eliminan esas filas.

In [ ]:
#Eliminar filas donde 'Total' sea 0
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Total'] != 0]

- Ahora, se hace una categorización más clara y concisa en *Tipo de núcleo familiar*:
    - Pareja casada.
    - Pareja no casada.
    - Familia monoparental

- Igualmente, se hará con *Número de hijos*. Se categorizará únicamente por número de hijos sin distinguir si son o no mayores de 25 años:
    - 0 hijos
    - 1 hijo
    - 2 o más hijos

In [ ]:
# Reemplazar valores en la columna "Tipo de núcleo familiar"
tipo_nucleo_familiar_df['Tipo de núcleo familiar'] = tipo_nucleo_familiar_df['Tipo de núcleo familiar'].replace({
    'Pareja casada, con o sin hijos convivientes': 'Pareja casada',
    'Pareja no casada, con o sin hijos convivientes': 'Pareja no casada',
    'Padre sólo con hijos convivientes': 'Familia monoparental',
    'Madre sóla con hijos convivientes': 'Familia monoparental'
})

tipo_nucleo_familiar_df['Número de hijos'] = tipo_nucleo_familiar_df['Número de hijos'].replace({
    '0 hijos conviviendo': '0 hijos', '0 hijos conviviendo menores de 25 años': '0 hijos',
    '1 hijo conviviendo': '1 hijo', '1 hijo conviviendo menor de 25 años': '1 hijo',
    '2 o más hijos conviviendo': '2 hijos o más', '2 hijos o más conviviendo menores de 25 años': '2 hijos o más'
})
# Verificar los cambios
print(tipo_nucleo_familiar_df['Tipo de núcleo familiar'].unique())
print(tipo_nucleo_familiar_df['Número de hijos'].unique())

- Agrupamos columnas con *Provincia*s, *Tipo de núcelo familiar* y *Número de hijos* iguales.

In [ ]:
#reset_index() es utilizado para convertir la Serie generada por groupby en un dataframe.
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df.groupby(['Provincias', 'Tipo de núcleo familiar', 'Número de hijos'])['Total'].sum().reset_index()

In [ ]:
tipo_nucleo_familiar_df.head(18)

- Siguiendo con el procesamiento de *tipo_nucleo_familiar_df* se va a redimensionar el dataframe. Pues lo que se pretende es tener sólamente una fila por provincia. Entonces, se harán diferentes columnas para las combinaciones de valores de *Tipo de núcleo familiar* y *Número de hijos*.
    - Además, dimensionando por columnas, tendremos una mayor comprensión de los valores que toma el dataframe.

In [ ]:
# Crear una nueva columna combinando 'Tipo de núcleo familiar' y 'Número de hijos'
tipo_nucleo_familiar_df['Categoria'] = tipo_nucleo_familiar_df['Tipo de núcleo familiar'] + ' con ' + tipo_nucleo_familiar_df['Número de hijos']

# Usar pivot para reorganizar la tabla sin incluir la categoría en el índice
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df.pivot(index='Provincias', columns='Categoria', values='Total')

# Para tener un DataFrame "normal", reiniciamos el índice
tipo_nucleo_familiar_df.reset_index(inplace=True)

# Eliminamos 'Categoría' como nombre de las columnas
tipo_nucleo_familiar_df.columns.name = None

# Mostrar el resultado
tipo_nucleo_familiar_df.head()

- Por último, se va a calcular el índice de cada tipo de familia en función del total por provincia.
    - Se suma los valores de cada provincia (excepto columna *Provincias*).
    - Tipo de familia / total
    - Se actualiza el dataframe

In [ ]:
# Calculamos el total por provincia
tipo_nucleo_familiar_df["Total"] = tipo_nucleo_familiar_df.iloc[:, 1:].sum(axis=1)

# Calculamos el índice de cada tipo de familia
for column in tipo_nucleo_familiar_df.columns[1:-1]:  # Excluimos 'Provincias' y 'Total'
    tipo_nucleo_familiar_df[column] = (
        tipo_nucleo_familiar_df[column] / tipo_nucleo_familiar_df["Total"]
    )

# Eliminamos la columna "Total" ya que no la necesitamos más
tipo_nucleo_familiar_df.drop(columns=["Total"], inplace=True)

# Mostrar el DataFrame actualizado
tipo_nucleo_familiar_df.head()

### dispositivos_renovable_df

In [ ]:
dispositivos_renovable_df.head(20)

- La columna *Ingresos Netos* no se tendrá en cuenta, pues no aportan interés al no especificarse a qué tipo de núcleo familiar corresponde tales ingresos. 
- Por lo tanto, sólo se agrupa por provincia y por disposición de dispositivos de energía renovable.

In [ ]:
# Agrupar por provincia (groupby) y sumar las viviendas con y sin dispositivos
dispositivos_renovable_df = dispositivos_renovable_df.groupby(['Provincias', 'Dispone de dispositivo de energía renovable'])['Total'].sum().unstack()

In [ ]:
dispositivos_renovable_df.head()

- A continuación, se cambia los valores totales de dispone de dispositivos de energía renovable por el índice que representan en función al total de dispositivos.

In [ ]:
# Calcular el total de dispositivos por provincia
dispositivos_renovable_df["Total"] = dispositivos_renovable_df["No"] + dispositivos_renovable_df["Si"]

# Calcular el índice de dispositivos de energía renovable por provincia
dispositivos_renovable_df["Si"] = dispositivos_renovable_df["Si"] / dispositivos_renovable_df["Total"]
dispositivos_renovable_df["No"] = dispositivos_renovable_df["No"] / dispositivos_renovable_df["Total"]
# Eliminar la columna "Total", só es necesaria para el cálculo del índice
dispositivos_renovable_df.drop(columns=["Total"], inplace=True)
   

#Mostrar resultado
dispositivos_renovable_df.head()

- Crear dataframe con los resultados obtenidos:
- dispositivos_renovable_df:
    - Provincias: nombre de la provincia.
    - Dispositivos de energía renovable: Índice de viviendas con dispositivos de energía renovable (sobre 1).

In [ ]:
# Reiniciar el índice para tener Provincias como columna.
dispositivos_renovable_df.reset_index(inplace=True)

# Eliminar la columna "No", ya que no es necesaria, pues se considera únicamente dispositivos de energía renovable.
dispositivos_renovable_df.drop(columns=["No"], inplace=True) 

# Cambiar el nombre de la columna "Si" a "Dispositivos de energía renovable" para mayor claridad.
dispositivos_renovable_df.rename(columns={'Si': 'Índice dispositivos energía renovable'}, inplace=True)

# Eliminar el nombre de las columnas
dispositivos_renovable_df.columns.name = None

# Mostrar resultado
dispositivos_renovable_df.head()

## Integrar datos

- Hacer merge de datos de los datos de porcentaje de dispositivos renovable y tipos de núcleo familiares en un mismo dataframe.

- Se combinan los datos de las columnas:
    - Provincias (dipositivos_renovable_df y tipo_nucelo_familiar_df)
    - Índice dispositivos energía renovable (dipositivos_renovable_df)
    - Todas referente a los índices de tipos de núcleos familiares (tipo_nucelo_familiar_df)

- Antes de juntar ambos datasets, se comprueba que contienen las mismas provincias.
    - Se comparan valores de provincias de cada datasets.
    - Se muestra resultados: *set()*: Conjunto vacío.
    - Se hace un print que expresa si tienen las mismas provincias.

In [ ]:
# Listar las provincias únicas de cada DataFrame
provincias_dispositivos = set(dispositivos_renovable_df['Provincias'].unique())
provincias_hogar = set(tipo_nucleo_familiar_df['Provincias'].unique())

# Provincias presentes en un DataFrame pero no en el otro
provincias_solo_dispositivos = provincias_dispositivos - provincias_hogar
provincias_solo_hogar = provincias_hogar - provincias_dispositivos

# Mostrar los resultados
print("Provincias en dispositivos_porcentaje_df pero no en media_hogar_df:")
print(provincias_solo_dispositivos)

print("\nProvincias en media_hogar_df pero no en dispositivos_porcentaje_df:")
print(provincias_solo_hogar)

# Comprobar si tienen valores exactamente iguales
valores_iguales = provincias_dispositivos == provincias_hogar
print("\n¿Ambos DataFrames tienen exactamente las mismas provincias?:", valores_iguales)


- Ambas dataframes contienen las mismas provincias, se continúa con el merge. Los provincias tienne el mismo orden en ambos dataframes. Por lo tanto, concatenándolas y eliminándo una columna Provincia, ya no se tendrá columnas replicadas.

In [ ]:
# Eliminar la columna 'Provincias' de tipo_nucleo_familiar_df pues ya está en dispositivos_renovable_df
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df.drop(columns=['Provincias'])

# Concatenar las DataFrames
tarjeta_3 = pd.concat([dispositivos_renovable_df, tipo_nucleo_familiar_df], axis=1)

- **Mostrar cómo queda el dataframe procesado para esta tarjeta de datos.**

In [ ]:
tarjeta_3.head() #mostrar cómo queda el dataframe final

- **Se guardan los datos como csv en carpeta data/gold**:


In [ ]:
with open(GOLD_DATA_PATH + "data_card_3_df.csv", "w") as f:
    tarjeta_3.to_csv(f, sep=';', encoding='utf-8', index=False)

## Profiling de la tarjeta para la hipótesis 3

- En este apartado, primeramente, se crea un Profile Report en formato html. Las conclusiones extraídas relevantes de este report se comentarán en el entregable 2.

In [ ]:
from  ydata_profiling import ProfileReport

profile = ProfileReport(tarjeta_3, title="Data Card 3 Profiling Report")
profile.to_file("../profiling/data_card_3_profile_report.html")

- A continuación, se procede con un profiling de los datos realizados aparte.

In [ ]:
tarjeta_3.info()